<a href="https://colab.research.google.com/github/disath1554/--10/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%A2%D0%B8%D1%82%D0%B0%D0%BD%D0%B8%D0%BA%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Решение задачи классификации на примере датасета Titanic
## Постановка задачи
Гибель Титаника является одним из самых печально известных кораблекрушений в истории. 15 апреля 1912 года во время своего первого плавания, Титаник затонул после столкновения с айсбергом, при этом погибло 1502 из 2224 пассажиров и членов экипажа.

Некоторые группы людей имели больше шансов выжить, по сравнению с другими. Например, женщины, дети, пассажиры высшего класса.

С помощью методов машинного обучения нужно предсказать, кто из пассажиров мог выжить в этой трагедии.

Для обучения применяются данные, размещенные на ресурсе Kaggle: <a href='https://www.kaggle.com/c/titanic'>Titanic: Machine Learning from Disaster | Kaggle.</a>

## Описание dataset Titanic
Набор данных включает в себя два CSV-файла. Файл `train.csv` представляет собой обучающий набор, файл `test.csv` — тестовый набор. Обучающий набор содержит признак `Survived` для каждого пассажира, обозначающий, выжил данный пассажир или нет (0 для умерших, 1 для выживших).

Каждая строчка наборов данных содержит следующие поля:

* `Pclass` — класс размещения (1, 2, 3);

* `Name` — имя;

* `Sex` — пол;

* `Age` — возраст;

* `SibSp` — количество братьев, сестер, сводных братьев, сводных сестер, супругов на борту титаника;

* `Parch` — количество родителей, детей (в том числе приемных) на борту титаника;

* `Ticket` — номер билета;

* `Fare` — плата за проезд;

* `Cabin` — каюта;

* `Embarked` — порт посадки (C — Шербур; Q — Квинстаун; S — Саутгемптон).

В поле `Age` приводится количество полных лет. Для детей меньше 1 года — дробное.

## Анализ данных

In [1]:
# импорт библиотек
import pandas as pd
import matplotlib.pyplot as plt

# для деления на выборки
from sklearn.model_selection import train_test_split


In [2]:
# чтение файла и вывод первых пяти строк таблицы
df = pd.read_csv('https://raw.githubusercontent.com/disath1554/ML/main/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# получение информации о файлe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Как видно из результата работы функции info() в столбцах Age, Cabin и Embarked &mdash; есть пропущенные значения.
<pre>
 5   Age          714 non-null    float64
 10  Cabin        204 non-null    object
 11  Embarked     889 non-null    object
</pre>

Пропущенные значения в столбце Embarked заменим модой S (наиболее часто встречаемым значением), пропущенные значения в столбце Age можно заменить средним значением по столбцу или медианным значением.


In [4]:
# подготовка признаков

# замена пропущенных значений в столбце Age, используйте fillna()
m_age =   df['Age'].mean()# вычисление медианы или среднего значения
df['Age'] = df['Age'].fillna(m_age)

# замена пропущенных значений в столбце Embarked, fillna()
df['Embarked'] = df['Embarked'].fillna('S')

#кодирование столбца Sex, метод apply()
df['Sex'] = df['Sex'].apply(lambda x: 1 if x == 'male' else 0)

In [5]:
# выделение признаков
X = df[['Pclass', 'Age', 'Sex', 'SibSp', 'Parch']] # признаки
y = df['Survived'] # целевой признак


In [6]:
# деление на выборки
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(623, 5) (268, 5) (623,) (268,)


# Модель DecisionTreeClassifier

In [7]:
# импорт модели
from sklearn.tree import DecisionTreeClassifier


# создание модели
dtc = DecisionTreeClassifier(max_depth=9)

# обучение модели
dtc.fit(x_train, y_train)

# модель готова предсказывать


DecisionTreeClassifier(max_depth=9)

In [ ]:
# оценка модели на тренировочных и тестовых данных
from sklearn.metrics import f1_score

y_pred_train = dtc.predict(x_train)
y_pred_test = dtc.predict(x_test)

print('F1 train =', f1_score(y_train, y_pred_train))
print('F1 test =', f1_score(y_test, y_pred_test))

F1 train = 0.8666666666666666
F1 test = 0.7393939393939394


__Задание.__
1. Создайте данные для новых пассажиров и получите предсказание.
Например, 17-летний пассажир (мужчина), путешествующий вторым классом без родственников.
2. Используя данные из файла `train` оцените вероятность, того, что ваши пассажиры смогли выжить.
3. Сравните вычисленные вероятности с результатом работы модели.
4. Сделайте предсказания для ваших пассажиров с помощью моделей логистической регрессии и k-ближайших соседей, сравните результаты.

In [ ]:
new_data = {'Pclass': [2, 1, 3],
            'Age': [17, 80, 10],
            'Sex' : [1, 1, 1],
            'SibSp': [0, 2, 1],
            'Parch': [1, 1, 1]}
new_data = pd.DataFrame(new_data)
y_pred_new = dtc.predict(new_data)
y_pred_new

array([0, 1, 1])

In [ ]:
k_pass = len(df[(df['Pclass']==2) & (df['Sex']==1) & (df['Age'] > 12) & (df['Age'] < 25)])
k_pass_v = len(df[(df['Pclass']==2) & (df['Sex']==1) & (df['Age'] > 12) & (df['Age'] < 25) & df['Survived']==1])
print(k_pass_v/k_pass)

0.045454545454545456


In [ ]:
a = df[(df['Pclass']==2) & (df['Sex']==1) & (df['Age'] > 12) & (df['Age'] < 25)]
a

# Логистическая регрессия

# kNN